<a href="https://colab.research.google.com/github/takacsanna/igy_neveld_fakt/blob/main/ora2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>